In [1]:
cd /content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data

/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data


In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('output_train_added.csv')
val_df = pd.read_csv('output_val_added.csv')

In [4]:
train_df = train_df.interpolate().fillna(0)
val_df = val_df.interpolate().fillna(0)

## **Near Fall 생성 (A_T1)**

In [5]:
W_METHOD_A = {
    'mean_joint_velocity': 0.3,
    'mean_angular_velocity': 0.4,
    'com_velocity': 0.3
}

In [6]:
def compute_instability_method_A(df, weights):
    score = (
        weights['mean_joint_velocity']   * df['mean_joint_velocity'] +
        weights['mean_angular_velocity'] * df['mean_angular_velocity'] +
        weights['com_velocity']          * df['com_velocity']
    )
    return score

train_df['instability_A'] = compute_instability_method_A(train_df, W_METHOD_A)
val_df['instability_A'] = compute_instability_method_A(val_df, W_METHOD_A)

In [7]:
def apply_threshold_global(df, score_col, percentile=95):
    th = np.percentile(df[score_col], percentile)

    near_fall = (
        (df['label'] == 0) &
        (df[score_col] >= th)
    ).astype(int)

    return near_fall, th

In [8]:
train_df['near_fall_A_T1'], th_A_T1 = apply_threshold_global(
    train_df,
    score_col='instability_A',
    percentile=95
)

val_df['near_fall_A_T1'], th_A_T1 = apply_threshold_global(
    val_df,
    score_col='instability_A',
    percentile=95
)

## **Near Fall 라벨 재정의**

In [9]:
train_df['label_orig'] = train_df['label']
val_df['label_orig'] = val_df['label']

In [10]:
def redefine_label_3class(df, near_fall_col):
    """
    return: np.ndarray of {0,1,2}
    """
    new_label = np.zeros(len(df), dtype=int)

    # 1. Fall
    new_label[df['label'] == 1] = 2

    # 2. Near-Fall (only from Normal)
    new_label[
        (df['label'] == 0) &
        (df[near_fall_col] == 1)
    ] = 1

    return new_label

train_df['label_3class'] = redefine_label_3class(
    train_df,
    near_fall_col='near_fall_A_T1'
)

val_df['label_3class'] = redefine_label_3class(
    val_df,
    near_fall_col='near_fall_A_T1'
)

In [11]:
train_df['label_3class'].value_counts().sort_index()

,count
label_3class,
0,381566
1,13010
2,20424


In [12]:
val_df['label_3class'].value_counts().sort_index()

,count
label_3class,
0,54296
1,1832
2,3072


In [13]:
# 기존 낙상 침범 여부
assert (train_df.loc[train_df['label'] == 1, 'label_3class'] == 2).all()
assert (val_df.loc[val_df['label'] == 1, 'label_3class'] == 2).all()

In [14]:
# Near-Fall이 비낙상에서만 생성되었는가
assert not (
    (train_df['label'] == 1) &
    (train_df['label_3class'] == 1)
).any()

assert not (
    (val_df['label'] == 1) &
    (val_df['label_3class'] == 1)
).any()

## **Build LSTM**

In [15]:
import torch
import random

def fix_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [16]:
# 가장 위험한 상태가 시퀀스의 Label
# Fall(2) > Near-Fall(1) > Normal(0)
def make_sequences_3class(df, window_size=30, step=1, label_col='label_3class', feature_cols=None):

    x, y = [], []

    for file, group in df.groupby("file"):
        data = group[feature_cols].values
        labels = group[label_col].values

        for i in range(0, len(data) - window_size + 1, step):
            seq = data[i:i+window_size]
            window_labels = labels[i:i+window_size]

            if 2 in window_labels:
                target = 2
            elif 1 in window_labels:
                target = 1
            else:
                target = 0

            x.append(seq)
            y.append(target)

    return np.array(x), np.array(y)

In [17]:
import torch.nn as nn
class LSTMModel(nn.Module):
    def __init__(self, input_size=51, hidden_size=64, num_classes=3):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])   # (B, 3)

In [18]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight

def train_model_3class(x, y, x_val, y_val, epochs=10, batch_size=64, lr=1e-3):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMModel(num_classes=3).to(device)

    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.array([0,1,2]),
        y=y
    )
    loss_fn = nn.CrossEntropyLoss(
        weight=torch.tensor(class_weights, dtype=torch.float).to(device)
    )

    optim = torch.optim.Adam(model.parameters(), lr=lr)

    train_ds = TensorDataset(
        torch.tensor(x).float(),
        torch.tensor(y).long()
    )
    val_ds = TensorDataset(
        torch.tensor(x_val).float(),
        torch.tensor(y_val).long()
    )

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for seq, label in train_loader:
            seq = seq.to(device)
            label = label.to(device)

            logits = model(seq)
            loss = loss_fn(logits, label)

            optim.zero_grad()
            loss.backward()
            optim.step()

            total_loss += loss.item()

        print(f"[Epoch {epoch+1}] loss={total_loss/len(train_loader):.4f}")

    return model

In [19]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

def evaluate_3class(model, x_val, y_val):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    preds = []

    loader = DataLoader(
        TensorDataset(torch.tensor(x_val).float()),
        batch_size=64,
        shuffle=False
    )

    with torch.no_grad():
        for seq, in loader:
            seq = seq.to(device)
            logits = model(seq)
            pred = torch.argmax(logits, dim=1)
            preds.extend(pred.cpu().numpy())

    preds = np.array(preds)
    y_val = np.array(y_val)

    # 핵심 지표
    f1_macro = f1_score(y_val, preds, average="macro", zero_division=0)
    f1_weighted = f1_score(y_val, preds, average="weighted", zero_division=0)
    precision_macro = precision_score(y_val, preds, average="macro", zero_division=0)
    recall_macro = recall_score(y_val, preds, average="macro", zero_division=0)

    print(classification_report(
        y_val, preds,
        target_names=["Normal","Near-Fall","Fall"]
    ))
    print(confusion_matrix(y_val, preds))

    return {
        "F1_macro": f1_macro,
        "F1_weighted": f1_weighted,
        "Precision_macro": precision_macro,
        "Recall_macro": recall_macro
    }

In [20]:
window_to_model = {}
results = []

FEATURE_COLS = [c for c in train_df.columns if (
    c.endswith('_x') or
    c.endswith('_y') or
    c.endswith('_score')
)]

In [21]:
WINDOW = 10
fix_seed(42)

x_train, y_train = make_sequences_3class(
    train_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

x_val, y_val = make_sequences_3class(
    val_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

model_3cls_10 = train_model_3class(
    x_train, y_train,
    x_val, y_val,
    epochs=10,
    batch_size=64,
    lr=1e-3
)

window_to_model[WINDOW] = model_3cls_10

metrics = evaluate_3class(model_3cls_10, x_val, y_val)

results.append({
    "Window": WINDOW,
    "Model": "LSTM_3Class",
    **metrics
})

[Epoch 1] loss=1.0635
[Epoch 2] loss=1.0373
[Epoch 3] loss=1.0406
[Epoch 4] loss=1.0701
[Epoch 5] loss=1.0700
[Epoch 6] loss=1.0597
[Epoch 7] loss=1.0484
[Epoch 8] loss=1.0401
[Epoch 9] loss=1.0355
[Epoch 10] loss=1.0353
              precision    recall  f1-score   support

      Normal       0.88      0.63      0.73     41765
   Near-Fall       0.07      0.00      0.00      6515
        Fall       0.19      0.81      0.31      5592

    accuracy                           0.57     53872
   macro avg       0.38      0.48      0.35     53872
weighted avg       0.71      0.57      0.60     53872

[[26192    70 15503]
 [ 2659     6  3850]
 [ 1059     4  4529]]


In [22]:
WINDOW = 20
fix_seed(42)

x_train, y_train = make_sequences_3class(
    train_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

x_val, y_val = make_sequences_3class(
    val_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

model_3cls_20 = train_model_3class(
    x_train, y_train,
    x_val, y_val,
    epochs=10,
    batch_size=64,
    lr=1e-3
)

window_to_model[WINDOW] = model_3cls_20

metrics = evaluate_3class(model_3cls_20, x_val, y_val)

results.append({
    "Window": WINDOW,
    "Model": "LSTM_3Class",
    **metrics
})

[Epoch 1] loss=1.0318
[Epoch 2] loss=1.0145
[Epoch 3] loss=1.0075
[Epoch 4] loss=1.0019
[Epoch 5] loss=0.9975
[Epoch 6] loss=0.9910
[Epoch 7] loss=0.9882
[Epoch 8] loss=0.9880
[Epoch 9] loss=0.9843
[Epoch 10] loss=0.9841
              precision    recall  f1-score   support

      Normal       0.84      0.71      0.77     31331
   Near-Fall       0.30      0.01      0.02      8237
        Fall       0.35      0.88      0.50      8384

    accuracy                           0.62     47952
   macro avg       0.50      0.53      0.43     47952
weighted avg       0.66      0.62      0.59     47952

[[22231   191  8909]
 [ 3170    87  4980]
 [ 1027    10  7347]]


In [23]:
WINDOW = 30
fix_seed(42)

x_train, y_train = make_sequences_3class(
    train_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

x_val, y_val = make_sequences_3class(
    val_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

model_3cls_30 = train_model_3class(
    x_train, y_train,
    x_val, y_val,
    epochs=10,
    batch_size=64,
    lr=1e-3
)

window_to_model[WINDOW] = model_3cls_30

metrics = evaluate_3class(model_3cls_30, x_val, y_val)

results.append({
    "Window": WINDOW,
    "Model": "LSTM_3Class",
    **metrics
})

[Epoch 1] loss=0.9811
[Epoch 2] loss=0.9398
[Epoch 3] loss=0.9317
[Epoch 4] loss=0.9212
[Epoch 5] loss=0.9146
[Epoch 6] loss=0.9121
[Epoch 7] loss=0.9068
[Epoch 8] loss=0.9029
[Epoch 9] loss=0.9033
[Epoch 10] loss=0.9018
              precision    recall  f1-score   support

      Normal       0.74      0.72      0.73     22486
   Near-Fall       0.53      0.02      0.04      8450
        Fall       0.51      0.91      0.65     11096

    accuracy                           0.63     42032
   macro avg       0.59      0.55      0.47     42032
weighted avg       0.64      0.63      0.57     42032

[[16299   109  6078]
 [ 4657   167  3626]
 [ 1000    40 10056]]


In [24]:
WINDOW = 40
fix_seed(42)

x_train, y_train = make_sequences_3class(
    train_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

x_val, y_val = make_sequences_3class(
    val_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

model_3cls_40 = train_model_3class(
    x_train, y_train,
    x_val, y_val,
    epochs=10,
    batch_size=64,
    lr=1e-3
)

window_to_model[WINDOW] = model_3cls_40

metrics = evaluate_3class(model_3cls_40, x_val, y_val)

results.append({
    "Window": WINDOW,
    "Model": "LSTM_3Class",
    **metrics
})

[Epoch 1] loss=0.8803
[Epoch 2] loss=0.8497
[Epoch 3] loss=0.8385
[Epoch 4] loss=0.8369
[Epoch 5] loss=0.8339
[Epoch 6] loss=0.8361
[Epoch 7] loss=0.8318
[Epoch 8] loss=0.8331
[Epoch 9] loss=0.8333
[Epoch 10] loss=0.8312
              precision    recall  f1-score   support

      Normal       0.77      0.00      0.00     14918
   Near-Fall       0.30      0.83      0.44      7738
        Fall       0.81      0.89      0.85     13456

    accuracy                           0.51     36112
   macro avg       0.63      0.57      0.43     36112
weighted avg       0.68      0.51      0.41     36112

[[   10 13333  1575]
 [    1  6451  1286]
 [    2  1476 11978]]


In [25]:
WINDOW = 50
fix_seed(42)

x_train, y_train = make_sequences_3class(
    train_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

x_val, y_val = make_sequences_3class(
    val_df,
    window_size=WINDOW,
    label_col='label_3class',
    feature_cols=FEATURE_COLS
)

model_3cls_50 = train_model_3class(
    x_train, y_train,
    x_val, y_val,
    epochs=10,
    batch_size=64,
    lr=1e-3
)

window_to_model[WINDOW] = model_3cls_50

metrics = evaluate_3class(model_3cls_50, x_val, y_val)

results.append({
    "Window": WINDOW,
    "Model": "LSTM_3Class",
    **metrics
})

[Epoch 1] loss=0.8183
[Epoch 2] loss=0.7505
[Epoch 3] loss=0.7482
[Epoch 4] loss=0.7384
[Epoch 5] loss=0.7423
[Epoch 6] loss=0.7357
[Epoch 7] loss=0.7358
[Epoch 8] loss=0.7393
[Epoch 9] loss=0.7356
[Epoch 10] loss=0.7289
              precision    recall  f1-score   support

      Normal       0.46      0.99      0.63      9838
   Near-Fall       0.01      0.00      0.00      6722
        Fall       0.97      0.60      0.74     13632

    accuracy                           0.59     30192
   macro avg       0.48      0.53      0.46     30192
weighted avg       0.59      0.59      0.54     30192

[[9759    4   75]
 [6533    3  186]
 [4944  531 8157]]


In [26]:
result_df = pd.DataFrame(results)
result_df

,Window,Model,F1_macro,F1_weighted,Precision_macro,Recall_macro
0,10,LSTM_3Class,0.346665,0.598725,0.380111,0.479319
1,20,LSTM_3Class,0.428760,0.593206,0.496415,0.532142
2,30,LSTM_3Class,0.474466,0.572129,0.593245,0.550296
3,40,LSTM_3Class,0.430973,0.411369,0.626621,0.574836
4,50,LSTM_3Class,0.456268,0.538910,0.478040,0.530263


## **Prediction**

In [ ]:
# !pip install ultralytics

In [27]:
LABEL = {
    0: "Normal",
    1: "Warning",
    2: "Fall"
}

In [29]:
import cv2
from ultralytics import YOLO

# ======================================
# YOLO 모델 로드
# ======================================
model = YOLO("yolov8s-pose.pt")
model.to("cuda")
model.fuse()  # 최적화

# keypoint 이름
kp_names = [
    "nose","left_eye","right_eye","left_ear","right_ear",
    "left_shoulder","right_shoulder","left_elbow","right_elbow",
    "left_wrist","right_wrist","left_hip","right_hip",
    "left_knee","right_knee","left_ankle","right_ankle"
]

# ======================================
# Skeleton batch 추출 함수
# ======================================
def extract_skeleton(video_path, batch_size=64):
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    batch = []
    frame_data = []

    prev = [0.0] * (len(kp_names) * 3)  # 이전 프레임 보정용

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        batch.append(frame)

        if len(batch) == batch_size:
            results = model(batch, verbose=False)
            for r in results:
                if r.keypoints is not None and len(r.keypoints.xy) > 0:
                    keypoints = r.keypoints.xy[0].cpu().numpy()
                    conf = r.keypoints.conf[0].cpu().numpy()
                    sk = []
                    for i in range(17):
                        x, y = keypoints[i]
                        c = conf[i]
                        sk += [float(x), float(y), float(c)]
                    prev = sk
                else:
                    sk = prev
                frame_data.append((frame_idx, sk))
                frame_idx += 1
            batch = []

    # 남은 프레임 처리
    if len(batch) > 0:
        results = model(batch, verbose=False)
        for r in results:
            if r.keypoints is not None and len(r.keypoints.xy) > 0:
                keypoints = r.keypoints.xy[0].cpu().numpy()
                conf = r.keypoints.conf[0].cpu().numpy()
                sk = []
                for i in range(17):
                    x, y = keypoints[i]
                    c = conf[i]
                    sk += [float(x), float(y), float(c)]
                prev = sk
            else:
                sk = prev
            frame_data.append((frame_idx, sk))
            frame_idx += 1

    cap.release()
    return frame_data  # [(frame_idx, [x1,y1,c1,...,x17,y17,c17]), ...]

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLOv8s-pose summary (fused): 81 layers, 11,615,724 parameters, 0 gradients, 30.2 GFLOPs


In [33]:
import cv2

skeleton_connections = [
    (0,1),(0,2),(1,3),(2,4),
    (0,5),(0,6),(5,7),(7,9),
    (6,8),(8,10),(5,6),(5,11),
    (6,12),(11,12),(11,13),(13,15),
    (12,14),(14,16)
]

def predict_video_lstm_3class(
    input_path,
    model,
    output_path,
    window_size=20
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    # 1) skeleton 추출
    frames_skel = extract_skeleton(input_path)
    sk_data = np.array([sk for _, sk in frames_skel])

    num_frames = len(sk_data)
    preds = np.zeros((num_frames, 3))  # (T, num_classes)

    # 2) sliding window 예측
    for i in range(num_frames - window_size + 1):
        seq = torch.tensor(
            sk_data[i:i+window_size]
        ).float().unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(seq)          # (1, 3)
            prob = torch.softmax(logits, dim=1).cpu().numpy()[0]

        preds[i + window_size - 1] = prob

    # 3) 비디오 쓰기
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret or frame_idx >= num_frames:
            break

        _, sk = frames_skel[frame_idx]
        sk = np.array(sk).reshape(-1, 3)

        # Skeleton draw
        for x, y, c in sk:
            cv2.circle(frame, (int(x), int(y)), 3, (0,255,0), -1)
        for i, j in skeleton_connections:
            x1, y1, _ = sk[i]
            x2, y2, _ = sk[j]
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

        # 예측 결과
        prob = preds[frame_idx]
        cls  = prob.argmax()
        label = LABEL[cls]

        color = (0,255,0)
        if label == "Warning":
            color = (0,255,255)
        elif label == "Fall":
            color = (0,0,255)

        cv2.putText(
            frame,
            f"{label} | N:{prob[0]:.2f} W:{prob[1]:.2f} F:{prob[2]:.2f}",
            (30, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            color,
            2
        )

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print("✅ Done:", output_path)

In [34]:
input_path = '/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Validation/01.원천데이터/VS/영상'

video_path = [
    '/Y/SY/00001_H_A_SY_C1/00001_H_A_SY_C1.mp4',
    '/Y/SY/00026_H_A_SY_C1/00026_H_A_SY_C1.mp4',
    '/Y/SY/00029_H_A_SY_C1/00029_H_A_SY_C1.mp4',
    '/Y/SY/00053_H_A_SY_C1/00053_H_A_SY_C1.mp4',
    '/Y/SY/00076_H_A_SY_C1/00076_H_A_SY_C1.mp4',
    '/N/N/00005_H_A_N_C1/00005_H_A_N_C1.mp4',
    '/N/N/00023_H_A_N_C1/00023_H_A_N_C1.mp4',
    '/N/N/00095_H_A_N_C1/00095_H_A_N_C1.mp4',
    '/N/N/00110_H_A_N_C1/00110_H_A_N_C1.mp4',
    '/N/N/00204_H_D_N_C1/00204_H_D_N_C1.mp4',
]

In [36]:
for path in video_path:
    print("🎥 Processing:", path)

    predict_video_lstm_3class(
        input_path + path,
        model_3cls_10,
        f"output_3cls_10_{path.split('/')[3]}.mp4",
        window_size=10
    )

🎥 Processing: /Y/SY/00001_H_A_SY_C1/00001_H_A_SY_C1.mp4
✅ Done: output_3cls_10_00001_H_A_SY_C1.mp4
🎥 Processing: /Y/SY/00026_H_A_SY_C1/00026_H_A_SY_C1.mp4
✅ Done: output_3cls_10_00026_H_A_SY_C1.mp4
🎥 Processing: /Y/SY/00029_H_A_SY_C1/00029_H_A_SY_C1.mp4
✅ Done: output_3cls_10_00029_H_A_SY_C1.mp4
🎥 Processing: /Y/SY/00053_H_A_SY_C1/00053_H_A_SY_C1.mp4
✅ Done: output_3cls_10_00053_H_A_SY_C1.mp4
🎥 Processing: /Y/SY/00076_H_A_SY_C1/00076_H_A_SY_C1.mp4
✅ Done: output_3cls_10_00076_H_A_SY_C1.mp4
🎥 Processing: /N/N/00005_H_A_N_C1/00005_H_A_N_C1.mp4
✅ Done: output_3cls_10_00005_H_A_N_C1.mp4
🎥 Processing: /N/N/00023_H_A_N_C1/00023_H_A_N_C1.mp4
✅ Done: output_3cls_10_00023_H_A_N_C1.mp4
🎥 Processing: /N/N/00095_H_A_N_C1/00095_H_A_N_C1.mp4
✅ Done: output_3cls_10_00095_H_A_N_C1.mp4
🎥 Processing: /N/N/00110_H_A_N_C1/00110_H_A_N_C1.mp4
✅ Done: output_3cls_10_00110_H_A_N_C1.mp4
🎥 Processing: /N/N/00204_H_D_N_C1/00204_H_D_N_C1.mp4
✅ Done: output_3cls_10_00204_H_D_N_C1.mp4


In [37]:
for path in video_path:
    print("🎥 Processing:", path)

    predict_video_lstm_3class(
        input_path + path,
        model_3cls_30,
        f"output_3cls_30_{path.split('/')[3]}.mp4",
        window_size=30
    )

🎥 Processing: /Y/SY/00001_H_A_SY_C1/00001_H_A_SY_C1.mp4
✅ Done: output_3cls_30_00001_H_A_SY_C1.mp4
🎥 Processing: /Y/SY/00026_H_A_SY_C1/00026_H_A_SY_C1.mp4
✅ Done: output_3cls_30_00026_H_A_SY_C1.mp4
🎥 Processing: /Y/SY/00029_H_A_SY_C1/00029_H_A_SY_C1.mp4
✅ Done: output_3cls_30_00029_H_A_SY_C1.mp4
🎥 Processing: /Y/SY/00053_H_A_SY_C1/00053_H_A_SY_C1.mp4
✅ Done: output_3cls_30_00053_H_A_SY_C1.mp4
🎥 Processing: /Y/SY/00076_H_A_SY_C1/00076_H_A_SY_C1.mp4
✅ Done: output_3cls_30_00076_H_A_SY_C1.mp4
🎥 Processing: /N/N/00005_H_A_N_C1/00005_H_A_N_C1.mp4
✅ Done: output_3cls_30_00005_H_A_N_C1.mp4
🎥 Processing: /N/N/00023_H_A_N_C1/00023_H_A_N_C1.mp4
✅ Done: output_3cls_30_00023_H_A_N_C1.mp4
🎥 Processing: /N/N/00095_H_A_N_C1/00095_H_A_N_C1.mp4
✅ Done: output_3cls_30_00095_H_A_N_C1.mp4
🎥 Processing: /N/N/00110_H_A_N_C1/00110_H_A_N_C1.mp4
✅ Done: output_3cls_30_00110_H_A_N_C1.mp4
🎥 Processing: /N/N/00204_H_D_N_C1/00204_H_D_N_C1.mp4
✅ Done: output_3cls_30_00204_H_D_N_C1.mp4
